In [ ]:
import pymc3 as pm
import numpy as np
import theano
import theano.tensor as tt
import arviz as az
import matplotlib.pyplot as plt

from tvb.simulator.lab import *
from tvb.simulator.backend.theano import TheanoBackend

from tvb_inversion.base.observation_models import linear

from tvb_inversion.pymc3.prior import Pymc3Prior
from tvb_inversion.pymc3.stats_model import Pymc3Model
from tvb_inversion.pymc3.inference import EstimatorPYMC
from tvb_inversion.pymc3.plot import plot_posterior_samples

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#conn = connectivity.Connectivity.from_file()
conn = connectivity.Connectivity()
conn.weights = np.array([[0., 2.], [2., 0.]])
conn.region_labels = np.array(["R1", "R2"])
conn.centres = np.array([[0.1, 0.1, 0.1], [0.2, 0.1, 0.1]])
conn.tract_lengths = np.array([[0., 2.5], [2.5, 0.]])
conn.configure()

sim = simulator.Simulator(
    model=models.oscillator.Generic2dOscillator(a=np.array([1.5])),
    connectivity=conn,
    coupling=coupling.Difference(),
    integrator=integrators.HeunStochastic(
        dt=1.0,
        noise=noise.Additive(
            nsig=np.array([1e-4]),
            noise_seed=42
        )
    ),
    monitors=[monitors.Raw()],
    simulation_length = 250
)

In [ ]:
sim.configure()

In [ ]:
sim.initial_conditions = np.zeros((conn.horizon, sim.model.nvar, conn.number_of_regions, 1))

In [ ]:
sim.configure()

In [ ]:
(t, X), = sim.run()

In [ ]:
X.shape

In [ ]:
plt.plot(X[:, 0, :, 0]);

In [ ]:
# Create theano backend functions
template_dfun = """
           import theano
           import theano.tensor as tt
           import numpy as np
           <%include file="theano-dfuns.py.mako"/>
           """
dfun = TheanoBackend().build_py_func(template_source=template_dfun, content=dict(sim=sim), 
                                     name="dfuns", print_source=True)

template_cfun = f"""
           import theano
           import theano.tensor as tt
           import numpy as np
           n_node = {sim.connectivity.number_of_regions}
           <%include file="theano-coupling.py.mako"/>
           """

cfun = TheanoBackend().build_py_func(template_source=template_cfun, content=dict(sim=sim), 
                                     name="coupling", print_source=True)

In [ ]:
def_std = 0.1

model = pm.Model()
with model:
    model_a_star = pm.Normal(name="model_a_star", mu=0.0, sd=1.0, shape=sim.model.a.shape)
    model_a = pm.Deterministic(name="model_a", var=sim.model.a * (1.0 + def_std * model_a_star))
    
    coupling_a_star = pm.Normal(name="coupling_a_star", mu=0.0, sd=1.0)
    coupling_a = pm.Deterministic(name="coupling_a", 
                                  var=sim.coupling.a[0].item() * (1.0 + def_std * coupling_a_star))
    
    x_init_star = pm.Normal(name="x_init_star", mu=0.0, sd=1.0, 
                            shape=sim.initial_conditions.shape[:-1])
    x_init = pm.Deterministic(name="x_init", 
                              var=sim.initial_conditions[:, :, :, 0] * (1.0 + def_std * x_init_star))
    
    BoundedNormal = pm.Bound(pm.Normal, lower=0.0)
    nsig_star = BoundedNormal(name="nsig_star", mu=0.0, sd=1.0)
    nsig = pm.Deterministic(name="nsig", 
                            var=sim.integrator.noise.nsig[0].item() * (1.0 + def_std * nsig_star))
    
    dWt_star = pm.Normal(name="dWt_star", mu=0.0, sd=1.0, shape=X.shape[:-1])
    
    amplitude_star = pm.Normal(name="amplitude_star", mu=0.0, sd=1.0)
    amplitude = pm.Deterministic(name="amplitude", var=1.0 * (1.0 + def_std * amplitude_star))

    offset_star = pm.Normal(name="offset_star", mu=0.0, sd=1.0)
    offset = pm.Deterministic(name="offset", var=def_std * offset_star)
    
    observation_noise_star = pm.HalfNormal("observation_noise_star", sd=1.0)
    observation_noise = pm.Deterministic("observation_noise", var=def_std * observation_noise_star)
    

In [ ]:
prior = Pymc3Prior(
    model=model,
    names=["model.a", "coupling.a", "x_init", "integrator.noise.nsig", "dWt_star", 
           "observation.model.amplitude", "observation.model.offset", "observation.noise"], 
    dist=[model_a, coupling_a, x_init, nsig, dWt_star, 
          amplitude, offset, observation_noise]
)

In [ ]:
def scheme(dWt, *x_prev):
    x_prev = x_prev[::-1]

    state = tt.stack(x_prev, axis=0)
    state = tt.transpose(state, axes=[1, 0, 2])

    cX = tt.zeros((sim.history.n_cvar, sim.history.n_node))
    cX = cfun(cX, sim.connectivity.weights, state, sim.connectivity.delay_indices,
              **prior.get_coupling_params())

    dX = tt.zeros((sim.model.nvar, sim.history.n_node))
    dX = dfun(dX, x_prev[0], cX, sim.model.spatial_parameter_matrix, **prior.get_model_params())
    
    return x_prev[0] + sim.integrator.dt * dX + dWt


In [ ]:
# model_builder = StochasticPymc3ModelBuilder(sim=sim, params=prior, 
#                                             observation_fun=linear, observation=X[:, :, :, 0])

pymc_model = Pymc3Model(sim=sim, params=prior)

In [ ]:
with stats_model.model:
    
    taps = list(-1 * np.arange(sim.connectivity.idelays.max() + 1) - 1)[::-1]
    x_sim, updates = theano.scan(
                    fn=scheme,
                    sequences=[dWt],
                    outputs_info=[dict(initial=x_init, taps=taps)],
                    n_steps=X.shape[0]
                )
    
    x_hat = pm.Deterministic(name="x_hat",
                             var=linear(x_sim, **prior.get_observation_model_params()))
    
    x_obs = pm.Normal(name="x_obs", mu=x_hat, sd=prior.dict.get("observation.noise", 1.0),
                      shape=X.shape[:-1], observed=X[:, :, :, 0])

In [ ]:
pymc_model.model

In [ ]:
pymc_estimator = EstimatorPYMC(stats_model=pymc_model)

In [ ]:
draws = 500
tune = 500
cores = 2

In [ ]:
inference_data = pymc_estimator.run_inference(draws, tune, cores, target_accept=0.9)

In [ ]:
init_params = {
    "model_a": sim.model.a,
    "coupling_a": sim.coupling.a[0],
    "nsig": sim.integrator.noise.nsig[0],
    "observation_noise": 0.0,
    "amplitude": 1.0,
    "offset": 0.0
}
plot_posterior_samples(pymc_estimator.inference_data, init_params)

In [ ]:
init_params = {
    "model_a_star": 0.0,
    "coupling_a_star": 0.0,
    "nsig_star": 0.0,
    "observation_noise_star": 0.0,
    "amplitude_star": 0.0,
    "offset_star": 0.0
}
plot_posterior_samples(pymc_estimator.inference_data, init_params)

In [ ]:
posterior_x_obs = pymc_estimator.inference_data.posterior_predictive.x_obs.values.reshape(
    (cores*draws, *X.shape[:-1]))

In [ ]:
f3, axes3 = plt.subplots(nrows=2, ncols=1, figsize=(18,15))
axes3[0].plot(np.percentile(posterior_x_obs[:, :, 0, 0], [2.5, 97.5], axis=0).T, 
              "k", label=r"$V_{95\% PP}(t)$")
axes3[0].plot(X[:, 0, 0, 0], label="V_observed")
axes3[0].legend(fontsize=16)
axes3[0].set_xlabel("time (ms)", fontsize=16)
axes3[0].tick_params(axis="both", labelsize=16)

axes3[1].plot(np.percentile(posterior_x_obs[:, :, 0, 1], [2.5, 97.5], axis=0).T, 
         "k", label=r"$W_{95\% PP}(t)$")
axes3[1].plot(X[:, 0, 1, 0], label="W_observed")
axes3[1].legend(fontsize=16)
axes3[1].set_xlabel("time (ms)", fontsize=16)
axes3[1].tick_params(axis="both", labelsize=16)

plt.show()

In [ ]:
pymc_estimator.inference_summary.loc[
    pymc_estimator.inference_summary.index.str.contains("model_a_star")].describe()

In [ ]:
pymc_estimator.inference_summary.loc[
    ["model_a_star", "coupling_a_star", "nsig_star", 
     "amplitude_star", "offset_star", "observation_noise_star"]]

In [ ]:
pymc_estimator.inference_summary.loc[
    pymc_estimator.inference_summary.index.str.contains("x_init_star")].describe()

In [ ]:
pymc_estimator.inference_summary.loc[
    pymc_estimator.inference_summary.index.str.contains("dWt_star")].describe()

In [ ]:
pymc_estimator.inference_data.to_netcdf(filename="pymc3_data/test1.nc", compress=False)